In [ ]:
import sys
!{sys.executable} -m pip install pandas numpy matplotlib scikit-learn seaborn umap-learn

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np
import matplotlib.pyplot as plt

# Forel 2024-25 Dataset Cleaning

# Further Analysis and Checks

# Analysis plots

# PCA plots

# t-SNE plots

# UMAP plots